## Importing Libraries

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

**2292. Products With Three or More Orders in Two Consecutive Years (Medium)**

**Table: Orders**

| Column Name   | Type |
|---------------|------|
| order_id      | int  |
| product_id    | int  |
| quantity      | int  |
| purchase_date | date |

order_id contains unique values.
Each row in this table contains the ID of an order, the id of the product purchased, the quantity, and the purchase date.
 
**Write a solution to report the IDs of all the products that were ordered three or more times in two consecutive years.**

Return the result table in any order.

The result format is shown in the following example.

**Example 1:**

**Input:** 

**Orders table:**

| order_id | product_id | quantity | purchase_date |
|----------|------------|----------|---------------|
| 1        | 1          | 7        | 2020-03-16    |
| 2        | 1          | 4        | 2020-12-02    |
| 3        | 1          | 7        | 2020-05-10    |
| 4        | 1          | 6        | 2021-12-23    |
| 5        | 1          | 5        | 2021-05-21    |
| 6        | 1          | 6        | 2021-10-11    |
| 7        | 2          | 6        | 2022-10-11    |

**Output:** 
| product_id |
|------------|
| 1          |

**Explanation:** 
- Product 1 was ordered in 2020 three times and in 2021 three times. Since it was ordered three times in two consecutive years, we include it in the answer.
- Product 2 was ordered one time in 2022. We do not include it in the answer.

In [0]:
orders_data_2292 = [
    (1, 1, 7, "2020-03-16"),
    (2, 1, 4, "2020-12-02"),
    (3, 1, 7, "2020-05-10"),
    (4, 1, 6, "2021-12-23"),
    (5, 1, 5, "2021-05-21"),
    (6, 1, 6, "2021-10-11"),
    (7, 2, 6, "2022-10-11"),
]

orders_columns_2292 = ["order_id", "product_id", "quantity", "purchase_date"]
orders_df_2292 = spark.createDataFrame(orders_data_2292,orders_columns_2292)
orders_df_2292.show()

+--------+----------+--------+-------------+
|order_id|product_id|quantity|purchase_date|
+--------+----------+--------+-------------+
|       1|         1|       7|   2020-03-16|
|       2|         1|       4|   2020-12-02|
|       3|         1|       7|   2020-05-10|
|       4|         1|       6|   2021-12-23|
|       5|         1|       5|   2021-05-21|
|       6|         1|       6|   2021-10-11|
|       7|         2|       6|   2022-10-11|
+--------+----------+--------+-------------+



In [0]:
orders_df_2292 = orders_df_2292\
                    .withColumn( "purchase_date", to_date("purchase_date"))

In [0]:
df_year_2292 = orders_df_2292.withColumn("year", year("purchase_date"))

In [0]:
df_counts_2292 = df_year_2292\
                    .groupBy("product_id", "year")\
                        .agg(countDistinct("order_id").alias("order_count")
                             )

In [0]:
df_filtered_2292 = df_counts_2292.filter("order_count >= 3")

In [0]:
windowSpec = Window.partitionBy("product_id").orderBy("year")

In [0]:
df_consec_2292 = df_filtered_2292.withColumn("prev_year", lag("year").over(windowSpec))

In [0]:
df_consec_2292\
    .filter(col("year") - col("prev_year") == 1)\
        .select("product_id")\
            .distinct().show()

+----------+
|product_id|
+----------+
|         1|
+----------+

